# Projeto

In [1]:
import pandas as pd
import re
from matplotlib.pyplot import figure, savefig, show
from dslabs_functions import plot_bar_chart

In [2]:
from pandas import DataFrame

file_tag = 'credit_score'
filename = 'class_credit_score.csv'
original_dataset: DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')

original_dataset.dtypes

Customer_ID                object
Month                      object
Name                       object
Age                        object
SSN                        object
Occupation                 object
Annual_Income             float64
Monthly_Inhand_Salary     float64
Num_Bank_Accounts           int64
Num_Credit_Card             int64
Interest_Rate               int64
NumofLoan                   int64
Type_of_Loan               object
Delay_from_due_date         int64
NumofDelayedPayment       float64
ChangedCreditLimit        float64
NumCreditInquiries        float64
CreditMix                  object
OutstandingDebt           float64
CreditUtilizationRatio    float64
Credit_History_Age         object
Payment_of_Min_Amount      object
TotalEMIpermonth          float64
Amountinvestedmonthly     float64
Payment_Behaviour          object
MonthlyBalance            float64
Credit_Score               object
dtype: object

*Tratar de valores de input errados*


In [3]:
import numpy as np

columns_to_treat = ["Age", "SSN", "Num_Bank_Accounts", "Interest_Rate", "NumofLoan", "Payment_of_Min_Amount", "MonthlyBalance"]
relevant = ["Age", "Num_Bank_Accounts", "Interest_Rate", "NumofLoan", "Payment_of_Min_Amount", "MonthlyBalance"]

df = pd.DataFrame(original_dataset)

df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Age'] = df['Age'].apply(lambda x: np.nan if x == -500 or x >= 150 else x)
df['Age'] = df['Age'].astype(str).str.replace('_', '', regex=False)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

# Check and clean 'SSN' column
pattern = r'^\d{3}-\d{2}-\d{4}$'  # Replace this with your SSN regex pattern
df['SSN'] = df['SSN'].apply(lambda x: x if re.match(pattern, str(x)) else np.nan)

df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].apply(lambda x: x if x >= 0 else np.nan)

#df['Interest_Rate'] = df['Interest_Rate'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan) TAXAS JURO ACIMA DE 30 E TAL IMPOSSIVEL

df['NumofLoan'] = df['NumofLoan'].apply(lambda x: x if x >= 0 else np.nan)

df['MonthlyBalance'] = df['MonthlyBalance'].apply(lambda x: x if x >= 0 else np.nan)

In [4]:
from dslabs_functions import get_variable_types

variables_types: dict[str, list] = get_variable_types(df)
numeric: list[str] = variables_types["numeric"]
print(numeric)

['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit', 'NumCreditInquiries', 'OutstandingDebt', 'CreditUtilizationRatio', 'TotalEMIpermonth', 'Amountinvestedmonthly', 'MonthlyBalance']


## Data Profiling

### Data Dimensionality

#### Figure 1 Nr Records x Nr numeric

*vale a pena experimentar com outros tipos de gráficos???*
- nao ha curse of dimensionality

In [ ]:
from matplotlib.pyplot import figure, savefig, show
from dslabs_functions import plot_bar_chart

figure(figsize=(4, 2))
values: dict[str, int] = {"nr records": df.shape[0], "nr numeric": df.shape[1]}
plot_bar_chart(
    list(values.keys()), list(values.values()), title="Nr of records vs nr numeric"
)
savefig(f"images/dim_dist/fig_1_records_variables.png")
show()

#### Figure 2 mr variables per type

*Rever tipos de daods da função get_variable_type*
- Trocar "Age" de simbólico para numérico

In [ ]:
df.dtypes

In [ ]:
from dslabs_functions import get_variable_types

variable_types: dict[str, list] = get_variable_types(df)

counts: dict[str, int] = {}
for tp in variable_types.keys():
    counts[tp] = len(variable_types[tp])

figure(figsize=(4, 2))
plot_bar_chart(
    list(counts.keys()), list(counts.values()), title="Nr of variables per type"
)
savefig(f"images/dim_dist/fig_2_variable_types.png")
show()

*Passar de object para category (simbolicas no panda)*

In [ ]:
symbolic: list[str] = variable_types["symbolic"]
df[symbolic] = df[symbolic].apply(lambda x: x.astype("category"))
df.dtypes

#### Figure 3 Nr missing values for dataset

In [ ]:
mv = {}

for var in df.columns:
    nr = df[var].isna().sum()
    print(var + " - " + str(nr))
    if nr > 0:
        mv[var] = nr

In [ ]:
mv: dict[str, int] = {}

for var in df.columns:
    nr: int = df[var].isna().sum()
    if nr > 0:
        mv[var] = nr

figure(figsize=(8, 4))
plot_bar_chart(
    list(mv.keys()),
    list(mv.values()),
    title="Nr of missing values per variable",
    xlabel="variables",
    ylabel="nr missing values",
)
savefig(f"images/dim_dist/figure_3_mv.png")
show()

*Valores nulos em percentagem(%)*

In [ ]:
mv: dict[str, int] = {}
for var in df.columns:
    nr: int = df[var].isna().sum()
    percentage = (nr / len(df[var]))
    if nr > 0:
        mv[var] = percentage

figure(figsize=(8, 4))
plot_bar_chart(
    list(mv.keys()),
    list(mv.values()),
    title="Nr of missing values per variable",
    xlabel="variables",
    ylabel="nr missing values",
    percentage=True
)
savefig(f"images/dim_dist/figure_3_mv_percentages.png")
show()

### Data Distribution

In [ ]:
summary5: DataFrame = df.describe(include='all')
summary5

#### Figure 4 Global Boxplots

*Não nos dá informação quase nenhuma*
- Boxplots are not applicable to non-numeric variables.

In [ ]:
from matplotlib.pyplot import savefig, show
from dslabs_functions import get_variable_types

variables_types: dict[str, list] = get_variable_types(df)
numeric: list[str] = variables_types["numeric"]
if [] != numeric:
    df[numeric].boxplot(rot=90)
    savefig(f"images/dim_dist/fig_4_global_boxplot.png")
    show()
else:
    print("There are no numeric variables.")

*Tirar 'Annual_Income' && 'TotalEMIpermonth' para ver se temos outra perspetiva*

In [ ]:
from matplotlib.pyplot import savefig, show
from dslabs_functions import get_variable_types

variables_types: dict[str, list] = get_variable_types(df)
numeric: list[str] = variables_types["numeric"]
if [] != numeric:
    excluded_indices = {1, len(numeric) - 3}
    numeric_excluded = [var for idx, var in enumerate(numeric) if idx not in excluded_indices]
    df[numeric_excluded].boxplot(rot=45)
    savefig(f"images/dim_dist/fig_4_global_boxplot_without_vars.png")
    show()
else:
    print("There are no numeric variables.")

#### Figure 6 Singe Variable Boxplots

In [ ]:
from numpy import ndarray
from matplotlib.figure import Figure
from matplotlib.pyplot import savefig, show, subplots
from dslabs_functions import define_grid, HEIGHT

if [] != numeric:
    rows: int
    cols: int
    rows, cols = define_grid(len(numeric))
    fig: Figure
    axs: ndarray
    fig, axs = subplots(
        rows, cols, figsize=(cols * HEIGHT, rows * HEIGHT), squeeze=False
    )
    i, j = 0, 0
    for n in range(len(numeric)):
        axs[i, j].set_title("Boxplot for %s" % numeric[n])
        axs[i, j].boxplot(df[numeric[n]].dropna().values)
        i, j = (i + 1, 0) if (n + 1) % cols == 0 else (i, j + 1)
    savefig(f"images/dim_dist/fig_6_single_boxplots.png")
    show()
else:
    print("There are no numeric numeric.")

*MonthlyBalance Missing Values removed to analyse the Boxplot better// Faz sentido? Fazer o mesmo para outras?*

#### Figure 8 Histograms

*Numericos*

In [ ]:
# 

# Granularity Numerical
import matplotlib.pyplot as plt
import dslabs_functions as ds
import re
import numpy as np

n_histograms = [10, 25, 50]

units = {
    "Age": "(Years)",
    "Annual_Income": "($)",
	"Monthly_Inhand_Salary": "($)",
	"Num_Bank_Accounts": "",
    "Num_Credit_Card": "",
	"Interest_Rate": "(%)",
	"NumofLoan": "",
	"Delay_from_due_date": "(days)",
    "NumofDelayedPayment": "",
	"ChangedCreditLimit": "",
	"NumCreditInquiries": "",
    "OutstandingDebt": "($)",
	"CreditUtilizationRatio": "",
	"TotalEMIpermonth": "($)",
    "Amountinvestedmonthly": "($)",
	"MonthlyBalance": "($)",
    "CreditHistoryAge": "(Years)"
}

hist_age = df["Credit_History_Age"]
hist_vals = []

for val in hist_age:
    matches = re.findall(r'(\d+) Years and (\d+) Months', str(val))
    if matches == []:
        hist_vals += [np.nan]
    else:
        hist_vals += [float(matches[0][0]) + ((float(matches[0][1])-1)/12) ,]

df["CreditHistoryAge"] = hist_vals

t = df.select_dtypes(include=['number', 'float'])
t["Age"] = pd.to_numeric(df['Age'], errors='coerce')
variables = t.columns
rows, cols = ds.define_grid(len(variables))
fig, axs = plt.subplots(rows, cols, figsize=(cols*ds.HEIGHT, rows*ds.HEIGHT))
i, j = 0, 0
for n in range(len(variables)):
    axs[i, j].set_title('Histogram for %s' % variables[n])
    axs[i, j].set_xlabel(f'{variables[n]} {units[variables[n]]}')
    axs[i, j].set_ylabel('Number of Records')

    # Adjusting the rwidth parameter to increase the distance between bars along the x-axis
    axs[i, j].hist(t[variables[n]].values, "auto", color='lightblue', rwidth=0.9)  # Adjust rwidth

    i, j = (i + 1, 0) if (n + 1) % cols == 0 else (i, j + 1)

savefig(f"images/dim_dist/fig_8_histogram_numeric.png")  # Save the figure
show()

*Simbólicos*

In [ ]:
# Select columns with numerical and string data types
from matplotlib import pyplot as plt
import dslabs_functions as ds

string_cols = df.select_dtypes(include="object").columns
symbolic_cols = [col for col in string_cols if col not in ["ID", "Name", "Customer_ID", "SSN", "Age", "Credit_History_Age"]]

# Combine numerical and string columns
rows, cols = ds.define_grid(len(symbolic_cols))

print(cols * ds.HEIGHT, rows * ds.HEIGHT)
fig, axs = plt.subplots(rows, cols, figsize=(cols * ds.HEIGHT, rows * ds.HEIGHT))
i, j = 0, 0
plt.subplots_adjust(hspace=1)

            
types = df["Type_of_Loan"]

types_values = []
for text in types:
    matches = re.findall(r'([\w\s-]+)(?:, | and |$)', str(text))
    m_filtered = [m if "and" not in m else m[4:] for m in matches]
    types_values += m_filtered

final = pd.Series(types_values).value_counts()

axs[i, j].set_title('Histogram for %s' % "Type_of_Loan")
axs[i, j].set_xlabel("Type_of_Loan")
axs[i, j].set_ylabel('Number of Records')
axs[i, j].bar(final.index, final.values, color = "lightblue")
axs[i, j].set_xticklabels(final.index, rotation=90)
i, j = i, j+1

for col in symbolic_cols:
    if col != "Type_of_Loan":
        axs[i, j].set_title('Histogram for %s' % col)
        axs[i, j].set_xlabel(col)
        axs[i, j].set_ylabel('Number of Records')
        value_counts = df[col].value_counts()
        axs[i, j].bar(value_counts.index, value_counts.values, color='lightblue')
        axs[i, j].set_xticklabels(value_counts.index, rotation=90)
        i, j = (i + 1, 0) if (j + 1) % cols == 0 else (i, j + 1)

plt.savefig(f"images/dim_dist/fig_8_histogram_symbolic.png")  # Save the figure
plt.show()

In [ ]:
from pandas import Series

target = "Credit_Score"

values: Series = df[target].value_counts()
print(values)

figure(figsize=(4, 2))
plot_bar_chart(
    values.index.to_list(),
    values.to_list(),
    title=f"Target distribution (target={target})",
)
savefig(f"images/dim_dist/fig_12_class_distribution.png")
show()

*Try to fit different known distributions*

In [ ]:
# from matplotlib.pyplot import subplots
# from dslabs_functions import set_chart_labels
# from dslabs_functions import define_grid, HEIGHT
# from numpy import log
# from pandas import Series
# from scipy.stats import norm, expon, lognorm
# from matplotlib.axes import Axes
# from dslabs_functions import plot_multiline_chart

# def compute_known_distributions(x_values: list) -> dict:
#     distributions = dict()
#     # Gaussian
#     mean, sigma = norm.fit(x_values)
#     distributions["Normal(%.1f,%.2f)" % (mean, sigma)] = norm.pdf(x_values, mean, sigma)
#     # Exponential
#     loc, scale = expon.fit(x_values)
#     distributions["Exp(%.2f)" % (1 / scale)] = expon.pdf(x_values, loc, scale)
#     # LogNorm
#     sigma, loc, scale = lognorm.fit(x_values)
#     distributions["LogNor(%.1f,%.2f)" % (log(scale), sigma)] = lognorm.pdf(
#         x_values, sigma, loc, scale
#     )
#     return distributions


# def histogram_with_distributions(ax: Axes, series: Series, var: str):
#     values: list = series.sort_values().to_list()
#     ax.hist(values, 20, density=True)
#     distributions: dict = compute_known_distributions(values)
#     plot_multiline_chart(
#         values,
#         distributions,
#         ax=ax,
#         title="Best fit for %s" % var,
#         xlabel=var,
#         ylabel="",
#     )

# if [] != numeric:
#     rows: int
#     cols: int
#     rows, cols = define_grid(len(numeric))
#     fig, axs = subplots(
#         rows, cols, figsize=(cols * HEIGHT, rows * HEIGHT), squeeze=False
#     )
#     i: int
#     j: int
#     i, j = 0, 0
#     for n in range(len(numeric)-14):
#         histogram_with_distributions(axs[i, j], df[numeric[n]].dropna(), numeric[n])        
#         i, j = (i + 1, 0) if (n + 1) % cols == 0 else (i, j + 1)
#     show()

#### Figure 10 Outlier Study

#### Figure 12 Class Distribution